In [ ]:
import os
import asyncio
from nano_graphrag import GraphRAG
import networkx as nx

# ✅ STEP 1 – Carica il testo da un file
with open("../data/alleanze.txt", "r", encoding="utf-8") as f:
    text = f.read()

# ✅ STEP 2 – Imposta la tua chiave API OpenAI
os.environ["OPENAI_API_KEY"] ="API_KEY"  # Sostituisci con la tua chiave reale

# ✅ STEP 3 – Inizializza GraphRAG
graphrag = GraphRAG(
    working_dir="./nano_cache",
    enable_local=True,
    tokenizer_type="tiktoken",
    tiktoken_model_name="gpt-3.5",  # usa gpt-4o solo se abilitato nel tuo account
    # graph_cluster_algorithm=None  # evitiamo errore sul clustering
)

# ✅ STEP 4 – Inserisci il documento (solo in ambienti Jupyter: usa `await`)
await graphrag.ainsert(text)

# ✅ STEP 5 – Carica il grafo salvato da file
G = nx.read_graphml("./nano_cache/graph_chunk_entity_relation.graphml")

# ✅ STEP 6 – Stampa le triplette soggetto --[relazione]--> oggetto
print("\n📌 Triplette estratte:")
for u, v, data in G.edges(data=True):
    relation = data.get("description", "(nessuna relazione)")
    print(f"{u} --[{relation}]--> {v}")



INFO:nano-graphrag:Loading tokenizer: type='tiktoken', name='gpt-3.5'
INFO:nano-graphrag:Load KV full_docs with 3 data
INFO:nano-graphrag:Load KV text_chunks with 3 data
INFO:nano-graphrag:Load KV llm_response_cache with 15 data
INFO:nano-graphrag:Load KV community_reports with 5 data
INFO:nano-graphrag:Loaded graph from ./nano_cache/graph_chunk_entity_relation.graphml with 73 nodes, 49 edges
INFO:nano-vectordb:Load (72, 1536) data
INFO:nano-vectordb:Init {'embedding_dim': 1536, 'metric': 'cosine', 'storage_file': './nano_cache/vdb_entities.json'} 72 data
INFO:nano-graphrag:Writing graph with 73 nodes, 49 edges



📌 Triplette estratte:
"LA FRANCIA" --["La Francia and Il Regno Unito formed an alliance during World War II to combat shared enemies."]--> "IL REGNO UNITO"
"LA FRANCIA" --["La Francia and Gli Stati Uniti were allies during World War II."]--> "GLI STATI UNITI"
"LA FRANCIA" --["La Francia was part of Le forze alleate working together against the Axis powers."]--> "LE FORZE ALLEATE"
"IL REGNO UNITO" --["Il Regno Unito and Gli Stati Uniti were allies, collaborating during World War II."]--> "GLI STATI UNITI"
"IL REGNO UNITO" --["Il Regno Unito was part of Le forze alleate, collaborating with other Allied countries during World War II."]--> "LE FORZE ALLEATE"
"LA SECONDA GUERRA MONDIALE" --["La Germania was a major opposing force to the Allies during World War II."]--> "LA GERMANIA"
"LA SECONDA GUERRA MONDIALE" --["L'Unione Sovietica played a key role in the Allies' victory over Nazi Germany in World War II."]--> "L'UNIONE SOVIETICA"
"GLI STATI UNITI" --["Gli Stati Uniti were members of Le

In [ ]:
from neo4j import GraphDatabase
import networkx as nx
from openai import OpenAI
import os
from dotenv import load_dotenv

# ✅ Carica API Key
load_dotenv()
client = OpenAI(api_key=os.getenv("API_KEY"))


# ✅ 1️⃣ Funzione che chiama OpenAI per estrarre il predicato
def extract_predicate_llm(description):
    """
    Chiede a OpenAI di restituire SOLO il predicato in forma standardizzata.
    """
    prompt = f"""
    Dato il seguente testo che descrive la relazione tra due entità:

    "{description}"

    Estrai e restituisci SOLO il predicato come una parola MAIUSCOLA e in inglese, senza altre spiegazioni.
    Possibili esempi di predicati: ALLIED_WITH, ENEMY_OF, PART_OF, LOCATED_IN, RELATED_TO.

    Rispondi solo con il predicato.
    """

    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": "Sei un estrattore di relazioni. Rispondi solo con il predicato in una parola maiuscola."},
            {"role": "user", "content": prompt}
        ]
    )

    predicate = response.choices[0].message.content.strip().upper()
    return predicate


# ✅ 2️⃣ Carica il grafo GraphML generato da GraphRAG
G = nx.read_graphml("./nano_cache/graph_chunk_entity_relation.graphml")


# ✅ 3️⃣ Estrai le triplette (soggetto, predicato, oggetto) usando LLM
triples = []
for u, v, data in G.edges(data=True):
    subject = u.replace('"', '')
    obj = v.replace('"', '')
    raw_description = data.get("description", "").replace('"', '')

    if not raw_description.strip():
        continue  # salta archi vuoti

    # Chiedi il predicato a OpenAI
    predicate = extract_predicate_llm(raw_description)
    triples.append((subject, predicate, obj))

print("\n✅ Triplette normalizzate:")
for t in triples:
    print(t)


# ✅ 4️⃣ Connessione a Memgraph
driver = GraphDatabase.driver("bolt://localhost:7687", auth=("", ""))  # Memgraph di default senza auth


# ✅ 5️⃣ Inserimento delle triplette in Memgraph
with driver.session() as session:
    for s, r, o in triples:
        session.run("""
            MERGE (a:Entity {name: $s})
            MERGE (b:Entity {name: $o})
            MERGE (a)-[:RELATION {type: $r}]->(b)
        """, s=s, o=o, r=r)

driver.close()
print("\n✅ Triplette inserite in Memgraph con predicato estratto automaticamente da OpenAI!")



INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


✅ Triplette normalizzate:
('LA FRANCIA', 'ALLIED_WITH', 'IL REGNO UNITO')
('LA FRANCIA', 'ALLIED_WITH', 'GLI STATI UNITI')
('LA FRANCIA', 'PART_OF', 'LE FORZE ALLEATE')
('IL REGNO UNITO', 'ALLIED_WITH', 'GLI STATI UNITI')
('IL REGNO UNITO', 'PART_OF', 'LE FORZE ALLEATE')
('LA SECONDA GUERRA MONDIALE', 'ENEMY_OF', 'LA GERMANIA')
('LA SECONDA GUERRA MONDIALE', 'PLAYED', "L'UNIONE SOVIETICA")
('GLI STATI UNITI', 'MEMBER_OF', 'LE FORZE ALLEATE')
('LA GERMANIA', 'ENEMY_OF', "L'UNIONE SOVIETICA")
('LA GERMANIA', 'OPPOSED', 'LE FORZE ALLEATE')
("L'UNIONE SOVIETICA", 'JOINED', 'LE FORZE ALLEATE')
('LE FORZE ALLEATE', 'ENEMY_OF', 'LA GERMANIA NAZISTA')
('GERMANIA NAZISTA', 'ALLIED_WITH', 'ITALIA FASCISTA')
('GERMANIA NAZISTA', 'ALLIED_WITH', 'GIAPPONE IMPERIALE')
('GERMANIA NAZISTA', 'ENEMY_OF', 'UNIONE SOVIETICA')
('GERMANIA NAZISTA', 'ALLIED_WITH', 'ROMANIA')
('GERMANIA NAZISTA', 'LEADER_OF', 'ADOLF HITLER')
('GERMANIA NAZISTA', 'ALLIED_WITH', 'PATTO D’ACCIAIO')
('GERMANIA NAZISTA', 'MARKED'

In [24]:
from neo4j import GraphDatabase

# ✅ Connessione a Memgraph
driver = GraphDatabase.driver("bolt://localhost:7687", auth=("", ""))

# ✅ Query corretta
with driver.session() as session:
    result = session.run("MATCH (a)-[r]->(b) RETURN a.name, r.type, b.name LIMIT 10")
    for record in result:
        print(f"{record['a.name']} --[{record['r.type']}]--> {record['b.name']}")

driver.close()


LA FRANCIA --[ALLIED_WITH]--> IL REGNO UNITO
LA FRANCIA --[ALLIED_WITH]--> GLI STATI UNITI
IL REGNO UNITO --[ALLIED_WITH]--> GLI STATI UNITI
LA FRANCIA --[PART_OF]--> LE FORZE ALLEATE
IL REGNO UNITO --[PART_OF]--> LE FORZE ALLEATE
GLI STATI UNITI --[MEMBER_OF]--> LE FORZE ALLEATE
LA GERMANIA --[OPPOSED]--> LE FORZE ALLEATE
L'UNIONE SOVIETICA --[JOINED]--> LE FORZE ALLEATE
LA SECONDA GUERRA MONDIALE --[ENEMY_OF]--> LA GERMANIA
LA SECONDA GUERRA MONDIALE --[PLAYED]--> L'UNIONE SOVIETICA


In [ ]:
from neo4j import GraphDatabase
from openai import OpenAI
import os
from dotenv import load_dotenv

load_dotenv()

# ✅ Nuova API OpenAI
client = OpenAI(api_key=os.getenv("API_KEY")) 

# ✅ Connessione a Memgraph
driver = GraphDatabase.driver("bolt://localhost:7687", auth=("", ""))

# ✅ Query per ottenere i dati
with driver.session() as session:
    result = session.run("""
        MATCH (a)-[r]->(b) 
        RETURN a.name, r.type, b.name
    """)
    
    context = "\n".join([
        f"{record['a.name']} {record['r.type']} {record['b.name']}" 
        for record in result
    ])

driver.close()

# ✅ Query con nuova API OpenAI
response = client.chat.completions.create(
    model="gpt-4",
    messages=[
        {"role": "system", "content": "Rispondi solo in base al contesto fornito."},
        # {"role": "user", "content": f"Contesto:\n{context}\n\nDomanda: Quali alleanze ha stretto la Francia?"}
        {"role": "user", "content": f"Contesto:\n{context}\n\nDomanda: Gli Stati Uniti erano alleati del Giappone?"}
    ]
)

print("Risposta LLM:")
print(response.choices[0].message.content)


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Risposta LLM:
No, gli Stati Uniti non erano alleati del Giappone.
